In [1]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import torch
import random
from transformers import GPT2TokenizerFast
from transformers import GPT2LMHeadModel

backbone = "its5Q/rugpt3large_mailqa"
device = "cpu"
model = GPT2LMHeadModel.from_pretrained(backbone).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(backbone, use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
!pip install pyowm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pyowm
from pyowm import OWM
from pyowm.utils.config import get_default_config
import os
import time

In [7]:
def ans(question, description=''):
    seed = random.randint(1, 10000000)
    print(f'Seed: {seed}')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    inp = tokenizer.encode(f'Ответ:',return_tensors="pt").to(device)
    print(f'question: {question}')
    gen = model.generate(inp, do_sample=True, top_p=0.9, temperature=0.76, max_new_tokens=50, repetition_penalty=1.2) #, stop_token="<eos>")
    
    gen = tokenizer.decode(gen[0])
    return gen[:gen.index('<eos>') if '<eos>' in gen else len(gen)]

In [8]:
def weather(city: str):
    owm = OWM('b59f0ea60a9a0709c51c7ed2acf1e36d')
    mgr = owm.weather_manager()
    observation = mgr.weather_at_place(city)
    weather = observation.weather_at_place(city)
    temperature = weather.temperature('celsius')

In [9]:
print(ans("зачем человеку кот?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Seed: 6035767
question: зачем человеку кот?
Ответ: О чём ты? о чем речь? 
О том, что кто-то не любит? 



In [10]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from telebot.util import MAX_MESSAGE_LENGTH
from logging import StreamHandler
from transformers.utils.dummy_flax_objects import FlaxAutoModelForSequenceClassification
from time import sleep
import telebot, wikipedia, re
import random
from telebot import types

from pyowm import OWM

def weather(city: str):
    owm = OWM('b59f0ea60a9a0709c51c7ed2acf1e36d')
    mgr = owm.weather_manager()
    observation = mgr.weather_at_place(city)
    weather = observation.weather
    temperature = weather.temperature('celsius')
    return temperature

def get_weather(message):
    city = message.text
    w = weather(city)
    t = list(w.values())    
    bot.send_message(message.from_user.id, f'В городе {city} сейчас {t[0]} градусов по Цельсию \n\nДля продолжение введите /start (можно кликнуть прямо здесь)')

wikipedia.set_lang("ru")

def getwiki(message):
    try:
        s = message.text
        ny = wikipedia.page(s)
        wikitext=ny.content[:1000]
        wikimas=wikitext.split('.')
        wikimas = wikimas[:-1]
        wikitext2 = ''
        for x in wikimas:
            if not('==' in x):
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        bot.send_message(message.chat.id, f'{wikitext2}\n\nДля продолжение введите /start (можно кликнуть прямо здесь)')

    except Exception as e:
        return 'В энциклопедии нет информации об этом'

BOT_TOKEN = '5404217160:AAHxz1w7HWAjUJ8kpKAgDC_9dr9XtGNgjCc'
bot = telebot.TeleBot(BOT_TOKEN)

@bot.message_handler(commands=["start"])
def start(m, res=False):
        markup=types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)
        item1=types.KeyboardButton("Чат")
        item2=types.KeyboardButton("Погода")
        item3=types.KeyboardButton("Википедия")
        markup.add(item1)
        markup.add(item2)
        markup.add(item3)
        msg = bot.send_message(m.chat.id, 'Нажми: \n\nЧAT — чтобы поболтать со мной \n\nПОГОДА — чтобы узнать погоду в любом городе \n\nВИКИПЕДИЯ — чтобы узнать что-то новое \n\n\ после тесирование каждой функции введите /start а затем соответствующую кнопку меню',  reply_markup=markup)
        bot.register_next_step_handler(msg, switch)

def switch(message):
   if (message.text == 'Погода'):
      weather_start(message)
   elif (message.text == 'Википедия'):
      wiki_start(message)
   else:
     chat_start(message)

def chat_start(message):
  if message.text.lower() == "чат":
     bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEFUgFi18F3VfmxYKI_7w3n0zn8CsMYFQAC8RYAAqQ6QEmsJ_tDoRaAKikE')
     bot.send_message(message.from_user.id, "Поболтаем? \n\nЯ ещё молодой бот и бывает что мне нужно время чтобы ответить, пожалуйста подождите если я иногда задумаюсь. После завершение разговора введите команду  /start \n\n Спрашивайте.")

def weather_start(message):
     bot.send_message(message.from_user.id, 'Введите название города')
     bot.register_next_step_handler(message, get_weather)
 
def wiki_start(message):
     bot.send_message(message.from_user.id, 'Отправьте мне любое слово, и я найду его значение на Wikipedia.')
     bot.register_next_step_handler(message, getwiki)

@bot.message_handler(content_types=['text'])
def chat(message):
  if message.text.lower() == "чат":
     bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEFUgFi18F3VfmxYKI_7w3n0zn8CsMYFQAC8RYAAqQ6QEmsJ_tDoRaAKikE')
     bot.send_message(message.from_user.id, "Поболтаем? \n\nЯ ещё молодой бот и бывает что мне нужно время чтобы ответить, пожалуйста подождите если я иногда задумаюсь. После завершение разговора введите команду /start \n\n Спрашивайте.")
  elif message.text.lower() == "привет!":
     bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEFUl5i2BLflcaEcGPHoTh4mP6aYQxSDQAC_RMAAvcjgUoI63bxJR2BXSkE')
  elif message.text.lower() == "привет":
     bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEFUl5i2BLflcaEcGPHoTh4mP6aYQxSDQAC_RMAAvcjgUoI63bxJR2BXSkE')
  elif message.text.lower() == "пока!":
     bot.send_message(message.from_user.id, "До свидания!")
     bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEFUgNi18HCpyD6gbuQGV9hZ6Ztrz7vEgACNhUAArByQEnnJNKG7RdNXSkE')
  elif message.text.lower() == 'пока':
     bot.send_message(message.from_user.id, "До свидания!")
     bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEFUgNi18HCpyD6gbuQGV9hZ6Ztrz7vEgACNhUAArByQEnnJNKG7RdNXSkE') 
  else:      
     text = ' '.join(message.text.split())
     print(text)

     reply = ans(text, "")

     bot.send_message(message.chat.id, reply)
     #logger.info(f'{reply}')

bot.polling(none_stop=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ты давно здесь находишься?
Seed: 9313828
question: ты давно здесь находишься?
